# Interoperability

The core tenant of InteropRouter is to allow for seamless interoperability between AI providers.

InteropRouter is based on the [OpenAI Responses API](https://platform.openai.com/docs/api-reference/responses) as its common interface as it supports the most features across providers. See [Migrate to the Responses API](https://platform.openai.com/docs/guides/migrate-to-responses) for more details.

The `Router` is the central abstraction. You register provider clients once, then make calls using a unified API. The router dispatches to the correct provider based on the model name.


In [1]:
import os

from anthropic import AsyncAnthropic
from google import genai
from openai import AsyncOpenAI

from interop_router.router import Router
from interop_router.types import RouterResponse

router = Router()
openai_client = AsyncOpenAI()
gemini_client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))
anthropic_client = AsyncAnthropic()
router.register("openai", openai_client)
router.register("gemini", gemini_client)
router.register("anthropic", anthropic_client)

## Strong Typing

InteropRouter expects you to use OpenAI Responses API types directly. For example, use `EasyInputMessageParam` from `openai.types.responses` to construct simple user and assistant messages.

These types are wrapped in `ChatMessage` for flexibility to store data necessary for interoperability, then translated to each provider's native format by the router.


In [2]:
from openai.types.responses import EasyInputMessageParam

from interop_router.types import ChatMessage

messages: list[ChatMessage] = []
user_message = """Classify the text into one of the provided categories and return the answer into the following JSON format: {"category": "CATEGORY"}
TEXT: Samba 3.8B, a simple Mamba+Sliding Window Attention architecture. And it has an infinite context length with linear complexity.
CATEGORIES: Unsupervised Learning, Statistics, SLM, LLM, Computer Vision, Reinforcement Learning
Think hard, but only return the JSON object as specified above."""
messages.append(ChatMessage(message=EasyInputMessageParam(role="user", content=user_message)))

## Optimal Translation

InteropRouter automatically translates parameters seamlessly between providers.

For example, we can use the OpenAI reasoning settings, but that will be translated to the corresponding parameter for Gemini and Anthropic. This allows you to use nearly the same code and your knowledge of the Responses API across providers.


In [3]:
openai_response = await router.create(
    input=messages,
    model="gpt-5.2",
    reasoning={"effort": "medium", "summary": "auto"},
    include=["reasoning.encrypted_content"],
    max_output_tokens=120_000,
)

gemini_response = await router.create(
    input=messages,
    model="gemini-3-flash-preview",
    reasoning={"effort": "low", "summary": "auto"},  # This maps to high reasoning effort in Gemini
    include=[
        "reasoning.encrypted_content"
    ],  # This sets include_thoughts=True in the Gemini ThinkingConfig which is necessary to get reasoning content
    max_output_tokens=120_000,
)

anthropic_response = await router.create(
    input=messages,
    model="claude-haiku-4-5-20251001",
    reasoning={"effort": "medium", "summary": "auto"},
    include=["reasoning.encrypted_content"],
    max_output_tokens=64_000,
)

## Response Structure

`RouterResponse.output` contains a list of `ChatMessage` objects with provider-normalized content. Since InteropRouter converts all provider responses to the OpenAI Responses API format, you can use the same extraction logic regardless of which provider generated the response.


In [4]:
def extract_text(response: RouterResponse) -> str:
    texts: list[str] = []
    for chat_message in response.output:
        if chat_message.message.get("type") == "message":
            content = chat_message.message.get("content")
            if isinstance(content, list):
                for c in content:
                    text = c.get("text", "")
                    if text:
                        texts.append(text)
    return "\n".join(texts)


generated_text = (
    "Model 1: "
    + extract_text(openai_response)
    + "\nModel 2: "
    + extract_text(gemini_response)
    + "\nModel 3: "
    + extract_text(anthropic_response)
)
print(generated_text.strip())

Model 1: {"category":"SLM"}
Model 2: {"category": "SLM"}
Model 3: ```json
{"category": "SLM"}
```


In [5]:
user_message_2 = (
    "These are three different model responses:\n"
    + generated_text.strip()
    + "\nPlease make a final determination of the category."
)

assistant_message = EasyInputMessageParam(
    role="user",
    content=user_message_2,
    type="message",
)
messages.append(ChatMessage(message=assistant_message))

gemini_response_2 = await router.create(
    input=messages,
    model="gemini-3-pro-preview",
    reasoning={"effort": "high", "summary": "auto"},
    include=["reasoning.encrypted_content"],
    max_output_tokens=120_000,
)

In [6]:
# Get the reasoning summary
summary = ""
for message in gemini_response_2.output:
    if message.message.get("type") == "reasoning":
        for summary_content in message.message.get("summary", []):
            summary += summary_content.get("text", "")

print("REASONING SUMMARY:\n" + summary.strip())
print("RESPONSE:\n" + extract_text(gemini_response_2).strip())

REASONING SUMMARY:
**Thinking Through the Classification**

Okay, so I've got this text snippet: "Samba 3.8B, a simple Mamba+Sliding Window Attention architecture. And it has an infinite context length with linear complexity."  My goal is to classify it into one of these six categories: Unsupervised Learning, Statistics, SLM, LLM, Computer Vision, or Reinforcement Learning.

First, I need to break down the text. "Samba 3.8B" screams AI model to me, with 3.8B suggesting size – likely parameters. "Mamba+Sliding Window Attention" strongly points towards language modeling, and "infinite context length with linear complexity" adds to this picture, alluding to efficient processing of long sequences.

Now, let's analyze the categories. Unsupervised Learning is a *method* of training, not the model itself, so that's out. Statistics is a field of math, again, not what this describes. Computer Vision is for images/video; Mamba is for sequences - so not that. Reinforcement Learning is about agent

## Easy Conversations

Each `ChatMessage` in `RouterResponse.output` has its `.message` field typed as `ResponseInputItemParam`. This means you can directly extend your conversation history with previous responses. Just append the messages from the previous response `messages.extend(response.output)` to continue the conversation.

The format was also designed to make it easy to serialize and deserialize to and from JSON for storage.


In [7]:
messages.extend(gemini_response_2.output)
messages

[ChatMessage(message={'role': 'user', 'content': 'Classify the text into one of the provided categories and return the answer into the following JSON format: {"category": "CATEGORY"}\nTEXT: Samba 3.8B, a simple Mamba+Sliding Window Attention architecture. And it has an infinite context length with linear complexity.\nCATEGORIES: Unsupervised Learning, Statistics, SLM, LLM, Computer Vision, Reinforcement Learning\nThink hard, but only return the JSON object as specified above.'}, id='8eb90298-fe8b-4eb9-987a-3b2f25c7f04f', timestamp=datetime.datetime(2025, 12, 23, 22, 10, 50, 488445, tzinfo=datetime.timezone.utc), created_by='user', interop={}, metadata={}, provider_kwargs={}, original_response=None),
 ChatMessage(message={'role': 'user', 'content': 'These are three different model responses:\nModel 1: {"category":"SLM"}\nModel 2: {"category": "SLM"}\nModel 3: ```json\n{"category": "SLM"}\n```\nPlease make a final determination of the category.', 'type': 'message'}, id='a8e37550-35b2-467

In [8]:
user_message_2 = """How does the architecture of SLMs typically differ from LLMs? Keep it brief."""
messages.append(ChatMessage(message=EasyInputMessageParam(role="user", content=user_message_2)))
openai_response_2 = await router.create(
    input=messages,
    model="gpt-5.1-codex-max",
    reasoning={"effort": "medium", "summary": "auto"},
    include=["reasoning.encrypted_content"],
    max_output_tokens=120_000,
)
print("RESPONSE:\n" + extract_text(openai_response_2).strip())
messages.extend(openai_response_2.output)

RESPONSE:
SLMs use fewer parameters and simplified or efficiency‑oriented mechanisms (like sparse or windowed attention) to run on limited hardware, whereas LLMs stack more layers and full self‑attention to maximize capacity and generality.


In [9]:
# Save the conversation to a file
from datetime import datetime
import json
from pathlib import Path

tmp_dir = Path.cwd() / "tmp" / "conversations"
tmp_dir.mkdir(parents=True, exist_ok=True)
chat_file = tmp_dir / f"chat_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"

chat_file.write_text(json.dumps([json.loads(m.model_dump_json()) for m in messages], indent=2))

27483

In [10]:
# Load the conversation from a file
loaded_messages: list[ChatMessage] = []
data = json.loads(chat_file.read_text())
for item in data:
    loaded_messages.append(ChatMessage.from_json(json.dumps(item)))
loaded_messages

[ChatMessage(message={'role': 'user', 'content': 'Classify the text into one of the provided categories and return the answer into the following JSON format: {"category": "CATEGORY"}\nTEXT: Samba 3.8B, a simple Mamba+Sliding Window Attention architecture. And it has an infinite context length with linear complexity.\nCATEGORIES: Unsupervised Learning, Statistics, SLM, LLM, Computer Vision, Reinforcement Learning\nThink hard, but only return the JSON object as specified above.'}, id='8eb90298-fe8b-4eb9-987a-3b2f25c7f04f', timestamp=datetime.datetime(2025, 12, 23, 22, 10, 50, 488445, tzinfo=datetime.timezone.utc), created_by='user', interop={}, metadata={}, provider_kwargs={}, original_response=None),
 ChatMessage(message={'role': 'user', 'content': 'These are three different model responses:\nModel 1: {"category":"SLM"}\nModel 2: {"category": "SLM"}\nModel 3: ```json\n{"category": "SLM"}\n```\nPlease make a final determination of the category.', 'type': 'message'}, id='a8e37550-35b2-467